In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np 
import pandas as pd 
import nltk
import nltk.corpus
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
data = pd.read_csv("/content/drive/MyDrive/vkr_mkdir/data_work")
data = data.sample(100000)

In [4]:
data = data[['overall','reviewerID','asin', 'reviewText']]
data

,overall,reviewerID,asin,reviewText
74455,5.0,AFSWI1KMDZIPS,0800141660,I can't believe I would ever say this but Hugh...
267194,5.0,A8LZ6DTLCIICF,6303206158,Good movie.
61953,5.0,A1L4RDILGM7D9E,0792834917,I have seen this movie many times but my husba...
348077,5.0,A1H6KSU4SK5Q40,B00RMJZXM2,"Wow, I really love these bulbs! Not too stark ..."
117779,5.0,A3TO1FAFI1MWU3,6300246795,One of the Best old Army movies and what great...
...,...,...,...,...
215248,5.0,A1K2DQS4C373GM,6302548748,I ordered the VHS version of this film which i...
370461,5.0,AGVB3UZIPFHEF,B000NIK9S2,Bought this for my girlfriend and she loves it...
92984,4.0,ABSWHASYB3W31,6300183785,When you( I ! ) buy an old TV show that I like...
117864,5.0,A11Y8MVERD1MHU,6300218406,Timerider: The Adventure of Lyle Swann is AWES...


In [7]:
from sklearn.model_selection import train_test_split
training_data, testing_data = train_test_split(
    data, test_size = 0.25, random_state = 42
)

In [8]:
training_data.shape, testing_data.shape

((75000, 4), (25000, 4))

In [9]:
training_data_top = training_data.groupby('asin').mean()
training_data_top = training_data_top.reset_index()

<ipython-input-9-b3053e0fbb17>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  training_data_top = training_data.groupby('asin').mean()


In [10]:
training_data_top.head()

,asin,overall
0,0001526863,5.000000
1,0005000009,2.833333
2,0005019281,4.434783
3,0005092663,5.000000
4,0005119367,4.698113


In [11]:
training_data_top_sort = training_data_top.sort_values(['overall', 'asin'], ascending=False)
training_data_top_sort.head()

,asin,overall
14799,B01HI9FU50,5.0
14797,B01HHNXZVS,5.0
14796,B01HH3964I,5.0
14795,B01HFKZQ42,5.0
14794,B01HFD5XMO,5.0


In [12]:
train_data_grped = training_data.groupby('asin')['overall'].count().sort_values(ascending=False)
train_data_grped.head(8)

asin
0790729628    444
0793906091    410
6301966554    295
6303042503    271
0792151712    263
0782010792    236
6302091632    221
0783225857    218
Name: overall, dtype: int64

In [13]:
avg_ratings_cnt = pd.DataFrame(training_data.groupby('asin')['overall'].mean())
avg_ratings_cnt['rating_cnts'] = pd.DataFrame(training_data.groupby('asin')['overall'].count())

In [14]:
avg_ratings_cnt

,overall,rating_cnts
asin,,
0001526863,5.000000,6
0005000009,2.833333,6
0005019281,4.434783,46
0005092663,5.000000,2
0005119367,4.698113,53
...,...,...
B01HI3VXWA,4.000000,1
B01HI9FU50,5.000000,1
B01HIWPUTI,3.000000,1


In [15]:
inference_df = testing_data[['reviewerID', 'asin', 'overall']]
inference_df.head(6)

,reviewerID,asin,overall
259198,A210UBLWQY7TZ7,6303149200,5.0
309915,AJWHHM6C82W8E,B00HMEJN2I,4.0
1224,A2T4IR9R88GE65,076700941X,5.0
133029,AQFPA14VXS7HS,6300988600,1.0
234379,A3HTULLL9180I2,6302990033,2.0
78175,AYPCKDWSH8545,1560689994,5.0


In [16]:
inference_df.rename(columns = {'overall' : 'ground_truth_ratings'}, inplace=True)

<ipython-input-16-a6cb44a8baf8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inference_df.rename(columns = {'overall' : 'ground_truth_ratings'}, inplace=True)


In [17]:
inference_df = inference_df.merge(training_data_top_sort, 
                        left_on='asin', 
                        right_on = 'asin')

In [18]:
inference_df.rename(columns = {'overall' : 'predicted_ratings'}, inplace = True)

In [19]:
inference_df

,reviewerID,asin,ground_truth_ratings,predicted_ratings
0,A210UBLWQY7TZ7,6303149200,5.0,4.054545
1,A3J0KKAM2QPX9F,6303149200,4.0,4.054545
2,A1JB7TD5JEP1TI,6303149200,2.0,4.054545
3,AWG2O9C42XW5G,6303149200,5.0,4.054545
4,A2II5MC1IT121I,6303149200,5.0,4.054545
...,...,...,...,...
22383,A1WB386MXOA5EE,B00T3DY8XK,5.0,4.000000
22384,A9UJXMP9JCDWW,B00006IDSO,5.0,4.000000
22385,AT49A45FKN1VW,B0052EK78O,2.0,4.750000
22386,A35TDGCC7CRNDJ,B00ODN0S70,5.0,4.833333


In [20]:
import sklearn.metrics as metric
MeanSquareError = metric.mean_squared_error(
    inference_df.ground_truth_ratings,
    inference_df.predicted_ratings
)

In [21]:
from math import sqrt
print(f"The Root Mean Square Error Value when avg rating is predicted for every product: {sqrt(MeanSquareError)}")

The Root Mean Square Error Value when avg rating is predicted for every product: 1.0607608097204855


In [22]:
reviewerID = data.groupby('reviewerID').count()
topReviewer = reviewerID[reviewerID.overall >= 3].index

In [23]:
topReviewers_ratings_df = data[data.reviewerID.isin(topReviewer)]

In [24]:
topReviewers_ratings_df.shape

(20354, 4)

In [25]:
data.shape

(100000, 4)

In [26]:
asinID = data.groupby('asin').count()
top_asins = asinID[asinID.overall >= 3].index
top_asin_ratings_df = topReviewers_ratings_df[
    topReviewers_ratings_df.asin.isin(top_asins)
]
top_asin_ratings_df.sort_values(by='overall', ascending=False).head(6)

,overall,reviewerID,asin,reviewText
238073,5.0,A5CYW5OL2T1J6,6303022928,"Great product, fast shipping!"
50737,5.0,A127ZVBDESPQ2N,079072961X,Excellence should be rewarded\nby the accolade...
38902,5.0,A2198O25QYUPXE,0790733919,There are many layers to this film and stoic L...
146697,5.0,A2D7NX9ZYBJZ13,6301465601,"I gotta admit it,I Couldn't find This Movie on..."
34826,5.0,A31SVABCAC038O,0788802194,Great original Disney movie
142226,5.0,A1UMW72H0X1AFO,6301239598,This blu-ray looks much better than the dvd fo...


In [27]:
top_asin_ratings_df.shape

(19442, 4)

In [28]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
# формат данных для него
import surprise
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV
from surprise import Dataset
from surprise import accuracy
from surprise import Reader

surprise_reader = Reader(rating_scale = (0.1, 5.0))
surprise_data = Dataset.load_from_df(
    top_asin_ratings_df[['reviewerID', 'asin', 'overall']],
    surprise_reader)

In [33]:
# Training the Model
from surprise.model_selection import train_test_split
training_set, testing_set = train_test_split(surprise_data, 
                                             test_size=0.25,
                                             random_state=42)

In [34]:
training_set.__class__, testing_set.__class__

(surprise.trainset.Trainset, list)

In [35]:
knn_model_10 = KNNWithMeans(k=10, min_k=6, 
                         sim_options={'name': 'pearson_baseline', 'user_based': True})
knn_model_10.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [36]:
knn_model_15 = KNNWithMeans(k=15, min_k=6, 
                         sim_options={'name': 'pearson_baseline', 'user_based': True})
knn_model_15.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [37]:
knn_model_20 = KNNWithMeans(k=20, min_k=6, 
                         sim_options={'name': 'pearson_baseline', 'user_based': True})
knn_model_20.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [39]:
from surprise import KNNBasic, SVD, NormalPredictor, KNNBaseline,KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering, Reader, dataset, accuracy
svd_model_50 = SVD(n_factors=50,reg_all=0.02)
svd_model_50.fit(training_set)

In [40]:
svd_model_25 = SVD(n_factors=25,reg_all=0.02)
svd_model_25.fit(training_set)

In [41]:
svd_model_75 = SVD(n_factors=75,reg_all=0.02)
svd_model_75.fit(training_set)

In [43]:
from prettytable import PrettyTable as PT
eval_table = PT()
# print(eval_table)
eval_table.field_names = ["Model Type", "RMSE Error"]
eval_table.add_row(["Avg Rating / Popularity Model", sqrt(MeanSquareError)])

In [44]:
# K-Nearest Neighbor
knn_10_test_predictions = knn_model_10.test(testing_set)
knn_15_test_predictions = knn_model_15.test(testing_set)
knn_20_test_predictions = knn_model_20.test(testing_set)

In [46]:
knn_10_test_predictions[1]

Prediction(uid='A3OUKV8KBGUPS4', iid='630165191X', r_ui=4.0, est=3.5, details={'actual_k': 0, 'was_impossible': False})

In [51]:
accuracy.rmse(knn_20_test_predictions)

RMSE: 1.0328


1.0328349107539707

In [48]:
eval_table.add_row(["KNN k=10", accuracy.rmse(knn_10_test_predictions)])
eval_table.add_row(["KNN k=15", accuracy.rmse(knn_15_test_predictions)])
eval_table.add_row(["KNN k=20", accuracy.rmse(knn_20_test_predictions)])

RMSE: 1.0328
RMSE: 1.0328
RMSE: 1.0328


In [52]:
# SVD
svd_25_test_predictions = svd_model_25.test(testing_set)
svd_50_test_predictions = svd_model_50.test(testing_set)
svd_75_test_predictions = svd_model_75.test(testing_set)

In [53]:
eval_table.add_row(["SVD n_factors=25", accuracy.rmse(svd_25_test_predictions)])
eval_table.add_row(["SVD n_factors=50", accuracy.rmse(svd_50_test_predictions)])
eval_table.add_row(["SVD n_factors=75", accuracy.rmse(svd_75_test_predictions)])

RMSE: 0.9505
RMSE: 0.9509
RMSE: 0.9550


In [54]:
print(eval_table)

+-------------------------------+--------------------+
|           Model Type          |     RMSE Error     |
+-------------------------------+--------------------+
| Avg Rating / Popularity Model | 1.0607608097204855 |
|            KNN k=10           | 1.0328332471729809 |
|            KNN k=15           | 1.032835107956328  |
|            KNN k=20           | 1.0328349107539707 |
|        SVD n_factors=25       | 0.9505251947858172 |
|        SVD n_factors=50       | 0.950904250117617  |
|        SVD n_factors=75       | 0.955034543980568  |
+-------------------------------+--------------------+


In [55]:
svd_param_grid = {
    'n_factors' : list(range(5, 100, 10)),
    'reg_all': [0.01, 0.02, 0.03]
}

In [56]:
svd_grid_search = GridSearchCV(
    SVD,
    svd_param_grid,
    measures = ['rmse'],
    cv = 5,
    refit = True
)

In [57]:
svd_grid_search.fit(surprise_data)

In [58]:
svd_grid_search.best_params

{'rmse': {'n_factors': 5, 'reg_all': 0.03}}

In [59]:
svd_cv_testing = svd_grid_search.test(testing_set)

In [60]:
accuracy.rmse(svd_cv_testing)

RMSE: 0.8115


0.8114735740961864

In [61]:
eval_table.add_row(["Best SVD n_factors=85", accuracy.rmse(svd_cv_testing)])

RMSE: 0.8115


In [62]:
print(eval_table)

+-------------------------------+--------------------+
|           Model Type          |     RMSE Error     |
+-------------------------------+--------------------+
| Avg Rating / Popularity Model | 1.0607608097204855 |
|            KNN k=10           | 1.0328332471729809 |
|            KNN k=15           | 1.032835107956328  |
|            KNN k=20           | 1.0328349107539707 |
|        SVD n_factors=25       | 0.9505251947858172 |
|        SVD n_factors=50       | 0.950904250117617  |
|        SVD n_factors=75       | 0.955034543980568  |
|     Best SVD n_factors=85     | 0.8114735740961864 |
+-------------------------------+--------------------+


In [63]:
from collections import defaultdict
def return_recommendations(inferences, num_rec = 5):
    """
    Method that takes in predictions and returns the top k similarly rated products.
    """
    most_similar_n = defaultdict(list)
    for user_id, item_id, user_rating, estimated_rating, _others in inferences:
        # storing the item_id and estimated_rating for each user_id
        most_similar_n[user_id].append((item_id, estimated_rating))
        
    # sorting and returning the top most recommendations
    for user_id, user_prod_ratings in most_similar_n.items():
        user_prod_ratings.sort(key = lambda prod: prod[1], reverse=True)
        # re assiging the top sorted products
        most_similar_n[user_id] = user_prod_ratings[:num_rec]
        
    return most_similar_n

most_similar_5 = return_recommendations(svd_cv_testing, num_rec = 5)

In [64]:
for user_id, user_ratings in most_similar_5.items():
    print("#---------------------------------")
    print(f"Products Recommended for Reviewer: {user_id}: \n")
    print([item_id for (item_id, item_rating) in user_ratings])  

Выходные данные были обрезаны до нескольких последних строк (5000).
#---------------------------------
Products Recommended for Reviewer: AEC90GPFKLAAW: 

['B000JFNO6I']
#---------------------------------
Products Recommended for Reviewer: A2R1HAXRNU0QX7: 

['6300247104']
#---------------------------------
Products Recommended for Reviewer: A2ZVERVFM25MJT: 

['6300147630']
#---------------------------------
Products Recommended for Reviewer: A303U1MOXG5R1F: 

['6302685141', '6302795567']
#---------------------------------
Products Recommended for Reviewer: A272RIGQ5ILO66: 

['6301971876']
#---------------------------------
Products Recommended for Reviewer: A10C4ZRZTWIV7Y: 

['6303082750']
#---------------------------------
Products Recommended for Reviewer: AJC7PEKZT4KFB: 

['6300214044', '0790737744', '630197204X', '6301802489', '630300203X']
#---------------------------------
Products Recommended for Reviewer: A2PBV1899CX3TE: 

['0792834976', '6303011969', '6300214710']
#-----------

In [67]:
data.query("reviewerID == 'AZFI8SNOTJBJY'")

,overall,reviewerID,asin,reviewText
170414,5.0,AZFI8SNOTJBJY,630196604X,I love this science fiction movie. It's an ol...
123899,5.0,AZFI8SNOTJBJY,6300268853,I love Lucy. I always have. She was special ...
42593,5.0,AZFI8SNOTJBJY,0790729989,I love horror films as well as crime stories a...
